# 3.0 ~ 3.2 

1. LLMs and Chat Models
2. Predict Messsages
3. Prompt Templates

In [11]:
# from dotenv import load_dotenv
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.5)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")

# load_dotenv()

prompt = template.format(country_a="France", country_b="Germany")

# llm = OpenAI(model_name="gpt-3.5-turbo-1106")

chat.predict(prompt)

'The distance between France and Germany varies depending on the specific locations being considered. However, the approximate driving distance between Paris, France and Berlin, Germany is around 1,050 kilometers (650 miles).'

In [ ]:
# a = llm.predict("How many planets are in the solar system?")
b = chat.predict("How many planets are in the solar system?")

In [ ]:
#a
b

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [7]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(
      content="You are a geography expert. And you only reply in Italian.",
    ),
  AIMessage(content="Ciao, mi chiamo Paolo"),
  HumanMessage(content="What is the distance between Mexico and the Thailand?"),
]

chat.predict_messages(messages)


AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

In [13]:
# Prompt Template

template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}?"),
  ]
)

prompt = template.format_messages(language="Greek", name="Socrates", country_a="Mexico", country_b="Thailand")

chat.predict_messages(prompt)

AIMessage(content='Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

## Note

프롬프트 템플릿 작성 후 format에서 placeholder에 값을 넣지 않으면 에러 발생

```python
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")

# load_dotenv()

# llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI(temperature=0.5)

template.format()
```

![스크린샷 2024-03-23 오후 7 54 14](https://github.com/Zamoca42/langchain-practice/assets/96982072/f551daff-fe48-4e35-9d73-2abb16d819b8)


# 3.3 

1. OutputParser and LCEL

In [16]:
# Output Parser는 텍스트로만 응답하는 LLM의 응답형태를 변경
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello,how, are, you?")

['Hello', 'how', 'are', 'you?']

In [25]:
template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a list generating machine. Everything you ar asked will be answered with a comma seperated list of max {max_items} in korean. Do NOT reply with anything else."),
    ("human", "{question}"),
  ]
)

prompt = template.format_messages(max_items=10, question="What are the planets")

result = chat.predict_messages(prompt)

print(result)

p = CommaOutputParser()

p.parse(result.content)

content='수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성'


['수성', '금성', '지구', '화성', '목성', '토성', '천왕성', '해왕성']

In [27]:
# chain 생성 prompt | model | output_parser

chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['피카츄', '파이리', '꼬부기', '이상해씨', '리자드왕.']